# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU


### I. Chuẩn bị môi trường:

In [100]:
import sqlite3
import pandas as pd
import json

In [101]:
def fetch_value(sqlCommand):
    conn = sqlite3.connect("data.db",check_same_thread=False)
    ret = pd.read_sql_query(sqlCommand, conn)
    conn.close()
    return ret

# Query dữ liệu
def query_sql(sql:str):
    listData = fetch_value(sql)
    return listData

# Lấy bài viết theo website.
def get_bai_viet(website:str):
    return query_sql(f"select * from baiviet where website='{website}'")

class Utils:
    def convertHMtoFloat(HM):
        h,m = HM.split(":")
        return int(h)+(float(m)/60)

### I. Tiền xử lý dữ liệu

 #### Yêu cầu: đưa các field về dạng chuẩn chung:

1. active_time: thời gian hoạt động, đưa về dạng sau:

```
    {
        <tên ngày trong tuần>:[ <thời gian mở cửa>,<thời gian đóng cửa> ]
    }

    *lowercase
    *thời gian đưa về float: 8h30 => 8.5
```
    
- ví dụ:

```
    {
        "thứ 2": [7,21.5],
        ...,
        "chủ nhật": [8,22.3]
    }
```

2. other_service: các dịch vụ khác, đưa về dạng sau:

```
    [
        <tên dịch vụ>,
        <tên dịch vụ>,
        ...,
    ]
    
    *lowercase
```
    
- ví dụ:

```
    [
        "có wifi"
        "giao hàng",
        ...,
    ]
```

#### I.1 Foody

In [102]:
def preprocessing_Foody():
    # Code here
    data = get_bai_viet("Foody")
    active_time = data["active_time"]
    other_service = data["other_service"]
    format_time = []
    for time in active_time:
        #time = time.open + ',' + time.close
        time = json.loads(time)
        time_format = []
        time_format.append(time['open'])
        time_format.append(time['close'])
        x = {
            "thứ 2":time_format,
            "thứ 3":time_format,
            "thứ 4":time_format,
            "thứ 5":time_format,
            "thứ 6":time_format,
            "thứ 7":time_format,
            "chủ nhật":time_format,
            }
        time = json.dumps(x)
        format_time.append(time)
    data['active_time'] = format_time
    return data

#### I.2 ShopeeFood

In [103]:
def preprocessing_ShopeeFood():
    def maping(name):
        if name=="monday":
            return "thứ 2"
        if name=="tuesday":
            return "thứ 3"
        if name=="wednesday":
            return "thứ 4"
        if name=="thursday":
            return "thứ 5"
        if name=="friday":
            return "thứ 6"
        if name=="saturday":
            return "thứ 7"
        if name=="sunday":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            open,close = at_js[key].split("-")
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("ShopeeFood")
    data["active_time"] = data["active_time"].apply(convert_active_time)

    return data

#### I.3 GoogleReview

In [104]:
def preprocessing_GoogleReview():
    # Code here
    return pd.DataFrame()

#### I.4 Tổng hợp lại dữ liệu tiền xử lý

In [105]:
# DATA = pd.concat([preprocessing_Foody(),preprocessing_ShopeeFood(),preprocessing_GoogleReview()])
preprocessing_Foody()
# DATA.describe()

['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí', 'Tip cho nhân viên', 'Có xuất hóa đơn đỏ']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có phòng riêng', 'Có chỗ chơi cho trẻ em', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc', 'Có xuất hóa đơn đỏ', 'Có nhạc sống']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Có khu vực hút thuốc']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có chỗ chơi cho trẻ em', 'Có thẻ thành viên', 'Có xuất hóa đơn đỏ']
['Có wifi', 'Có máy lạnh & điều hòa', 'Có chỗ đậu ôtô', 'Có bàn ngoài trời', 'Có hỗ trợ hội thảo']
['Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có phòng riêng', 'Có khu vực hút thuốc', 'Có xuất hóa đơn đỏ']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có phòng riêng', 'Tip

[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Cho mua về', 'Trả bằng thẻ', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc']
['Có wifi', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Tip cho nhân viên', 'Có xuất hóa đơn đỏ']
['Có wifi']
[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Có phòng riêng']
[]
[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Có phòng riêng', 'Giữ xe máy miễn phí']
[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Nên đặt trước']
['Có wifi', 'Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Giữ xe máy miễn phí', 'Có xuất hóa đơn đỏ']
[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ']
['Có giao hàng', 'Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí']
['Có wifi', 'Có máy lạnh & điều hòa']
[]
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có bàn ngoài trời', 'Có phòng riêng', 'Có khu vực hút thuốc', 'Có xuất h

['Có wifi', 'Cho mua về', 'Trả bằng thẻ', 'Nên đặt trước', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc']
['Cho mua về']
['Cho mua về']
['Cho mua về', 'Giữ xe máy miễn phí']
['Nên đặt trước']
[]
['Có wifi', 'Có bàn ngoài trời']
['Cho mua về', 'Nên đặt trước']
[]
['Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí']
[]
['Cho mua về']
['Có wifi', 'Có giao hàng', 'Cho mua về', 'Có chỗ đậu ôtô', 'Giữ xe máy miễn phí', 'Có thẻ thành viên']
[]
['Có wifi', 'Có giao hàng', 'Giữ xe máy miễn phí']
['Có wifi', 'Cho mua về', 'Trả bằng thẻ', 'Giữ xe máy miễn phí']
['Có wifi', 'Có máy lạnh & điều hòa', 'Cho mua về', 'Giữ xe máy miễn phí']
[]
[]
['Có wifi', 'Trả bằng thẻ', 'Có chỗ đậu ôtô', 'Nên đặt trước', 'Có bàn ngoài trời', 'Có phòng riêng', 'Tip cho nhân viên', 'Có khu vực hút thuốc', 'Có xuất hóa đơn đỏ']
[]
['Có wifi', 'Cho mua về']
[]
['Có wifi', 'Nên đặt trước']
[]
['Cho mua về', 'Giữ xe máy miễn phí']
['Có wifi', 'Có máy lạnh & điều hòa', 'Có chỗ đậu ôtô']
['Có wifi', 'Cho mua về', 'Có

['Có wifi']
['Có wifi', 'Cho mua về', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc', 'Có xuất hóa đơn đỏ']
['Có giao hàng', 'Cho mua về']
['Cho mua về', 'Giữ xe máy miễn phí']
['Cho mua về', 'Có bàn ngoài trời']
['Cho mua về']
['Có wifi', 'Có giao hàng', 'Cho mua về', 'Giữ xe máy miễn phí']
['Có wifi', 'Có xuất hóa đơn đỏ']
['Có giao hàng', 'Cho mua về', 'Có bàn ngoài trời']
['Có wifi', 'Có máy lạnh & điều hòa', 'Có bàn ngoài trời']
['Có wifi', 'Có máy lạnh & điều hòa', 'Trả bằng thẻ', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc', 'Có chiếu bóng đá']
['Cho mua về']
['Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí']
['Cho mua về', 'Giữ xe máy miễn phí']
['Giữ xe máy miễn phí']
['Cho mua về', 'Có bàn ngoài trời']
['Có wifi', 'Có máy lạnh & điều hòa', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc']
['Cho mua về', 'Có bàn ngoài trời']
['Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí']
['Có wifi', 'Có máy lạnh & điều hòa']
['Có wifi'

['Có giao hàng', 'Nên đặt trước']
['Có giao hàng', 'Nên đặt trước']
['Có giao hàng', 'Cho mua về', 'Có bàn ngoài trời', 'Có khu vực hút thuốc']
['Có wifi', 'Có máy lạnh & điều hòa', 'Cho mua về', 'Giữ xe máy miễn phí']
[]
['Có giao hàng', 'Nên đặt trước']
['Cho mua về', 'Có bàn ngoài trời']
['Có giao hàng']
['Có wifi', 'Có giao hàng', 'Cho mua về', 'Có chỗ đậu ôtô', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc']
['Có giao hàng', 'Nên đặt trước']
['Cho mua về']
['Có giao hàng', 'Cho mua về']
['Cho mua về']
['Có giao hàng', 'Cho mua về']
['Có wifi', 'Có máy lạnh & điều hòa', 'Cho mua về', 'Giữ xe máy miễn phí']
['Có giao hàng', 'Cho mua về']
['Có wifi', 'Có giao hàng', 'Cho mua về', 'Có chỗ đậu ôtô', 'Có phòng riêng', 'Giữ xe máy miễn phí', 'Có chiếu bóng đá']
['Có giao hàng', 'Cho mua về', 'Có bàn ngoài trời']
['Có giao hàng', 'Cho mua về', 'Có bàn ngoài trời', 'Giữ xe máy miễn phí', 'Có khu vực hút thuốc', 'Có chiếu bóng đá']
['Có wifi', 'Cho mua về', 'Giữ xe máy miễn phí']
['Cho mua v

,website_id,website,url,full_name,phone,district,rate,rate_count,favorite,active_time,price_from,price_to,other_service
0,Foody_107609,Foody,https://www.foody.vn/ho-chi-minh/the-log-resta...,The LOG Restaurant,None,1,7.896,251,887,"{""th\u1ee9 2"": [18, 23], ""th\u1ee9 3"": [18, 23...",200000.0,1500000.0,"[""C\u00f3 wifi"", ""C\u00f3 m\u00e1y l\u1ea1nh &..."
1,Foody_91179,Foody,https://www.foody.vn/ho-chi-minh/nha-hang-sora...,Sorae Restaurant - Lounge,None,1,8.168,251,701,"{""th\u1ee9 2"": [11, 23], ""th\u1ee9 3"": [11, 23...",300000.0,1000000.0,"[""C\u00f3 wifi"", ""C\u00f3 m\u00e1y l\u1ea1nh &..."
2,Foody_20860,Foody,https://www.foody.vn/ho-chi-minh/eon-heli-bar-...,EON 52 Heli Bar - Bitexco Tower,None,1,7.822,110,306,"{""th\u1ee9 2"": [10, 2], ""th\u1ee9 3"": [10, 2],...",200000.0,550000.0,"[""C\u00f3 wifi"", ""C\u00f3 m\u00e1y l\u1ea1nh &..."
3,Foody_8,Foody,https://www.foody.vn/ho-chi-minh/buffet-windso...,Buffet Hải Sản - Café Central An Đông - Khách ...,None,5,7.224,185,364,"{""th\u1ee9 2"": [11, 22], ""th\u1ee9 3"": [11, 22...",449000.0,853000.0,"[""C\u00f3 wifi"", ""C\u00f3 m\u00e1y l\u1ea1nh &..."
4,Foody_94523,Foody,https://www.foody.vn/ho-chi-minh/an-duyen-nha-...,Bếp Xanh An Duyên - Nhà Hàng Chay,None,5,7.880,151,224,"{""th\u1ee9 2"": [10, 21], ""th\u1ee9 3"": [10, 21...",30000.0,250000.0,"[""C\u00f3 wifi"", ""C\u00f3 m\u00e1y l\u1ea1nh &..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022,Foody_999121,Foody,https://www.foody.vn/ho-chi-minh/nhom-nau-an-b...,Nhóm Nấu Ăn Ba Tu,None,thủ đức,0.000,0,0,"{""th\u1ee9 2"": [7, 21], ""th\u1ee9 3"": [7, 21],...",100000.0,500000.0,"[""N\u00ean \u0111\u1eb7t tr\u01b0\u1edbc""]"
3023,Foody_1032768,Foody,https://www.foody.vn/ho-chi-minh/hai-thuy-cate...,Hai Thụy Catering - Cơm Văn Phòng,None,bình thạnh,0.000,0,0,"{""th\u1ee9 2"": [8, 17], ""th\u1ee9 3"": [8, 17],...",30000.0,100000.0,"[""C\u00f3 giao h\u00e0ng"", ""N\u00ean \u0111\u1..."
3024,Foody_1000022520,Foody,https://www.foody.vn/ho-chi-minh/a-say-oc-hai-...,"A Sấy - Ốc, Hải Sản - Bến Vân Đồn",None,4,0.000,0,0,"{""th\u1ee9 2"": [15, 21], ""th\u1ee9 3"": [15, 21...",0.0,0.0,"[""Cho mua v\u1ec1"", ""N\u00ean \u0111\u1eb7t tr..."
3025,Foody_855723,Foody,https://www.foody.vn/ho-chi-minh/dich-vu-nau-a...,Dịch Vụ Nấu Ăn Hạnh Phúc,None,bình thạnh,0.000,0,0,"{""th\u1ee9 2"": [7, 23], ""th\u1ee9 3"": [7, 23],...",500000.0,5000000.0,"[""C\u00f3 giao h\u00e0ng"", ""N\u00ean \u0111\u1..."


### 2. Thống kê dữ liệu

// Mô tả ...

In [106]:
# Code here

###  Phần 3,4,5